# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.enable('data_server')
alt.themes.enable('opaque')

%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [30]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0
...,...,...,...,...,...,...,...
22831,Magallanes y la Antartica,12,Punta Arenas,12101.0,141984.0,2020-11-27,438.0
22832,Magallanes y la Antartica,12,Rio Verde,12103.0,211.0,2020-11-27,0.0
22833,Magallanes y la Antartica,12,San Gregorio,12104.0,681.0,2020-11-27,0.0
22834,Magallanes y la Antartica,12,Timaukel,12303.0,282.0,2020-11-27,0.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3978 entries, 374 to 4351
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   region      3978 non-null   object        
 1   fecha       3978 non-null   datetime64[ns]
 2   fallecidos  3978 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 124.3+ KB


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29
...,...,...,...
1661,50-59,2020-12-02,1547
1662,60-69,2020-12-02,3258
1663,70-79,2020-12-02,4339
1664,80-89,2020-12-02,3991


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos la fecha por cada grupo etáreo.

In [7]:
alt.Chart(fallecidos_etareo).mark_line().encode(
    alt.X('fecha', title='Fecha'),
    alt.Y('fallecidos', title='Fallecidos'),
    color=alt.Color('grupo_de_edad', legend=alt.Legend(title="Grupo Etareo"))
).properties(title='N° de fallecidos a la fecha por grupo etareo',width=800,
    height=300,)

alt.Chart(...)

**Comentarios:** Se aprecia que el mayor número de fallecidos es en el rango 70-79, esto puede ser causa de lo complicado de las enfermedades respiratorias a su edad, lo que no tiene sentido considerando que los grupos de 80 años en adelante tienen menor cantidad de fallecidos, pero quizas esto se debe a que son menor población en total, o tienen menor movilidad, por ende se contagian menos, y se mueren menos de ellos.

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [8]:
alt.Chart(activos).mark_bar().encode(
    y= alt.Y('poblacion', title='Población'),
 x=alt.X('comuna', title='Comuna',sort=None)
 
).properties(title='Población de todas las comunas en Chile')

alt.Chart(...)

**Comentarios:** Se puede apreciar que la distribución de población es muy variante a lo largo de las comunas de Chile, y esa variación tiende a ser uniforme(describiendolo de una forma).En algunas comunas del norte y cuarta región existe población relativamente grande, y las comunas de la región metropolitana concentran gran cantidad de la población.
<!-- Debido a que no se podia incluir la variable region o provincia se decidio este tipo de grafico como lo mejor posible respecto a lo solicitado. -->

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [9]:
# Configure common options
base = alt.Chart(fallecidos_etareo).encode(
    alt.X('fecha:O', scale=alt.Scale(paddingInner=0),title='Fecha'),
    alt.Y('grupo_de_edad:O', scale=alt.Scale(paddingInner=0),title='Grupo Etareo'),
).properties(
    width=8000,
    height=300,title='Evolución de fallecidos por grupo etareo'
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('fallecidos:Q',
        scale=alt.Scale(scheme='viridis'),
        legend=alt.Legend(direction='horizontal',title='N° de fallecidos')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='fallecidos:Q',
    color=alt.condition(
        alt.datum.num_cars > 500,
        alt.value('black'),
        alt.value('white')
    )
)

# Draw the chart
heatmap + text

alt.LayerChart(...)

**Comentarios:** Se aprecia que la evolución en el número de fallecidos de los grupos 60-69,70-79,80-89, es mas rápida que la de los demás grupos.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [10]:
eac=activos.groupby("fecha",as_index=False).agg({"casos_activos": "sum"})
econ=confirmados.groupby("fecha",as_index=False).agg({"casos_confirmados": "sum"})
emu=fallecidos.groupby("fecha",as_index=False).agg({"fallecidos": "sum"})
graf1=alt.Chart(econ).mark_line().encode(
    x=alt.X('fecha', title='Fecha'),
    y=alt.Y('casos_confirmados', title='Casos Confirmados'),
).properties(title='Evolución de casos confirmados a la fecha en Chile')
graf2=alt.Chart(emu).mark_line().encode(
    x=alt.X('fecha', title='Fecha'),
    y=alt.Y('fallecidos', title='Fallecidos'),
).properties(title='Evolución de fallecidos a la fecha en Chile')
graf3=alt.Chart(eac).mark_line().encode(
    x=alt.X('fecha', title='Fecha'),
    y=alt.Y('casos_activos', title='Casos Activos'),
).properties(title='Evolución de casos activos a la fecha en Chile')
graf1 | graf2 | graf3

alt.HConcatChart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [62]:
datostasadeincidencia=confirmados.groupby(["region","fecha"]).apply(lambda x: x["casos_confirmados"].sum()*100000/ x["poblacion"].sum()).reset_index()
datostasadeincidencia.rename(columns={0:'Ti'},inplace=True)
alt.Chart(datostasadeincidencia).mark_circle().encode(
    x=alt.X('fecha', title='Fecha'),
    y=alt.Y('Ti', title='Tasa de incidencia'),
    color=alt.Color('region',scale=alt.Scale(scheme="category20"), legend=alt.Legend(title="Región"))).properties(title='Tasa de incidencia durante el tiempo para las regiones en Chile',width=500,
    height=300).interactive()


alt.Chart(...)

**Comentarios:** Se aprecia que todas las regiones tienen un comportamiento similar, exceptuando or la región de magallanes que en septiembre se dispara.

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [71]:
c1e6=confirmados.query("fecha==['2020-04-13']")
g1e6=alt.Chart(c1e6).mark_circle().encode(
    x=alt.X('casos_confirmados', title='Casos Confirmados'),
    y=alt.Y('tasa', title='Tasa de incidencia'),
    color=alt.Color('region', legend=alt.Legend(title="Región"),scale=alt.Scale(scheme="category20")
)).properties(title='Tasa de incidencia durante el tiempo para las regiones en Chile',width=700,
    height=300)
c2e6=confirmados.query("fecha==['2020-11-06']")
g2e6=alt.Chart(c2e6).mark_square().encode(
    x=alt.X('casos_confirmados', title='Casos Confirmados'),
    y=alt.Y('tasa', title='Tasa de incidencia'),
    color=alt.Color('region', legend=alt.Legend(title="Región"),scale=alt.Scale(scheme="category20")
)).properties(title='Tasa de incidencia durante el tiempo para las regiones en Chile',width=700,
    height=300)
(g1e6+g2e6).resolve_scale(
    color='independent'
)

alt.LayerChart(...)

**Comentarios:** Se puede notar que hay un gran aumento de casos confirmados y tasa de inciddencia debido a que solo se pueden visualizar cuadrados, y los círculos están concentrados cerca del origen.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [50]:
alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha', title='Fecha'),
    y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de Chile',width=700,
    height=400)

alt.Chart(...)

In [51]:
E7AYP=activos.query("region=='Arica y Parinacota'")
E7TAR=activos.query("region=='Tarapaca'")
E7ANT=activos.query("region=='Antofagasta'")
E7ATA=activos.query("region=='Atacama'")
E7COQ=activos.query("region=='Coquimbo'")
E7VAL=activos.query("region=='Valparaiso'")
E7MET=activos.query("region=='Metropolitana'")
E7OHI=activos.query('region == "Del Libertador General Libertador Bernardo O\'Higgins"')
E7MAU=activos.query("region=='Maule'")
E7NUB=activos.query("region=='Nuble'")
E7BIO=activos.query("region=='Biobio'")
E7ARA=activos.query("region=='La Araucania'")
E7RIO=activos.query("region=='Los Rios'")
E7LAG=activos.query("region=='Los Lagos'")
E7AYS=activos.query("region=='Aysen'")
E7MAG=activos.query("region=='Magallanes y la Antartica'")
GAYPE7=alt.Chart(E7AYP).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Arica y Parinacota',width=200,
    height=200)
GTARE7=alt.Chart(E7TAR).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Tarapacá',width=200,
    height=200)
GANTE7=alt.Chart(E7ANT).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Antofagasta',width=200,
    height=200)
GATAE7=alt.Chart(E7ATA).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Atacama',width=200,
    height=200)
GCOQE7=alt.Chart(E7COQ).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Coquimbo',width=200,
    height=200)
GVALE7=alt.Chart(E7VAL).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Valparaíso',width=200,
    height=200)
GMETE7=alt.Chart(E7MET).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región Metropolitana',width=200,
    height=200)
GOHIE7=alt.Chart(E7OHI).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región Del Libertador General Libertador Bernardo O\'Higgins',width=200,
    height=200)
GMAUE7=alt.Chart(E7MAU).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región del Maule',width=200,
    height=200)
GNUBE7=alt.Chart(E7NUB).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Ñuble',width=200,
    height=200)
GBIOE7=alt.Chart(E7BIO).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región del Bio Bio',width=200,
    height=200)
GARAE7=alt.Chart(E7ARA).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de La Araucanía',width=200,
    height=200)
GRIOE7=alt.Chart(E7RIO).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Los Ríos',width=200,
    height=200)
GLAGE7=alt.Chart(E7LAG).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Los Lagos',width=200,
    height=200)
GAYSE7=alt.Chart(E7AYS).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Aysen',width=200,
    height=200)
GMAGE7=alt.Chart(E7MAG).mark_line().encode(
    x=alt.X('fecha', title='Fecha'), y=alt.Y('casos_activos', title='Casos Activos'),
    color=alt.Color('comuna', legend=alt.Legend(title="Comuna")
)).properties(title='Evolución de casos activos de todas las comunas de la región de Magallanes y la Antartica',width=200,
    height=200)
(GAYPE7|GTARE7|GANTE7|GATAE7).resolve_scale(
    color='independent'
) & (GCOQE7|GVALE7|GMETE7|GOHIE7).resolve_scale(
    color='independent'
)& (GMAUE7|GNUBE7|GBIOE7|GARAE7).resolve_scale(
    color='independent'
)& (GRIOE7|GLAGE7|GAYSE7|GMAGE7).resolve_scale(
    color='independent'
)


C:\Users\claud\miniconda3\envs\mat281v2\lib\site-packages\altair\utils\core.py:187: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  "Defaulting to nominal.".format(typ)


alt.VConcatChart(...)

**Comentarios:** Los principales pros tienen que ver con el tipo de gráfico, debido a que el que contiene todas las comunas en uno, permite tener la información de que comuna alcanzo el máximo entre todas durante el periodo medido,y estimar tambien el número de casos activos promedio aproximado de Chile, mientras los otros entregan información mas detallada que puede servir a la gente perteneciente a su correspondiente región para saber como va su comuna y región en general.
Como contras ocurre que el grafico que contiene todas las comunas es muy general, entonces encontrar el valor de alguna comuna en específico un día específico es imposible, en cambio el defecto de tenerlos por separado, es que si se quiere tener el máximo nacional y el valor aproximado hay que inspeccionar todas las regiones, que no son pocas y que tambien no se puede tener una noción a simple vista de como se ha comportado el país en general debido a que retener la información de mirar todos los gráficos es imposible.

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [53]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}

In [144]:
fallecidose8=fallecidos
for cont in region_names:
    fallecidose8['region'] = fallecidose8['region'].replace([cont],region_names[cont])
fallecidose8=fallecidose8.groupby(["region"]).apply(lambda x: x["fallecidos"].max()).reset_index()
fallecidose8.rename(columns={0:'Máximos fallecidos'},inplace=True)
confirmadose8=confirmados.groupby(["region"]).apply(lambda x: x["casos_confirmados"].max()).reset_index()
confirmadose8.rename(columns={0:'Máximo casos confirmados'},inplace=True)
activose8=activos.groupby(["region"]).apply(lambda x: x["casos_activos"].mean()).reset_index()
activose8.rename(columns={0:'Promedio Activos'},inplace=True)
g1e8=alt.Chart(fallecidose8).mark_bar().encode(
    x=alt.X('region', title='Región'), y=alt.Y('Máximos fallecidos', title='Máximos fallecidos'),
).properties(title='Máximos fallecidos por región Chile',width=450,
    height=200)
g2e8=alt.Chart(confirmadose8).mark_bar().encode(
    x=alt.X('region', title='Región'), y=alt.Y('Máximo casos confirmados', title='Máximos casos confirmados'),
).properties(title='Máximos casos confirmados por región Chile',width=450,
    height=200)
g3e8=alt.Chart(activose8).mark_bar().encode(
    x=alt.X('region', title='Región'), y=alt.Y('Promedio Activos', title='Promedio casos Activos'),
).properties(title='Promedio de casos activos por región Chile',width=450,
    height=200)
g1e8|g2e8|g3e8

alt.HConcatChart(...)

**Comentarios:** Se presentan los datos solicitados, en partícular no entendi a que se referia con cada promedio y máximo, si temporal, entre las comunas de la respectiva región o entre ambas variables, si que hice lo último.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [145]:
import geopandas as gpd
from pathlib import Path

In [181]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [147]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [148]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [188]:
fallecidose9=fallecidos
for cont in region_names:
    fallecidose9['region'] = fallecidose9['region'].replace([cont],region_names[cont])
confirmadose9a=confirmados.groupby(["region","fecha","region_id"]).apply(lambda x: x["casos_confirmados"].sum()*100000/ x["poblacion"].sum()).reset_index()
confirmadose9a.rename(columns={0:'Tasa de incidencia'},inplace=True)
confirmadose9a.rename(columns = {'region_id': 'codigo_region'}, inplace = True)
confirmadose9b=confirmados.groupby(["region","fecha","region_id"]).apply(lambda x: x["casos_confirmados"].sum()).reset_index()
confirmadose9b.rename(columns={0:'Casos Confirmados'},inplace=True)
confirmadose9b.rename(columns = {'region_id': 'codigo_region'}, inplace = True)
activose9a=activos.groupby(["region","fecha","codigo_region"]).apply(lambda x: x["poblacion"].sum()).reset_index()
activose9a.rename(columns={0:'Poblacion'},inplace=True)
activose9b=activos.groupby(["region","fecha","codigo_region"]).apply(lambda x: x["casos_activos"].sum()).reset_index()
activose9b.rename(columns={0:'Casos Activos'},inplace=True)
regiones91=regiones.rename(columns = {'codregion': 'codigo_region'})
regiones91=regiones91.drop(['Region','objectid','cir_sena','area_km','st_area_sh','st_length_'],axis=1)
casos_geo = (
    confirmadose9a.merge(confirmadose9b,
        how="left",
        on=['codigo_region','fecha','region']).merge(activose9a,
        how="left",
        on=['codigo_region','fecha','region']).merge(activose9b,
        how="left",
        on=['codigo_region','fecha','region']).merge(fallecidose9,
        how="left",
        on=['fecha','region']).merge(regiones91,
        how="left",
        on=['codigo_region'])
)
casos_geo=casos_geo.loc[lambda x: x.notnull().all(axis=1)]
casos_geo.head()

,region,fecha,codigo_region,Tasa de incidencia,Casos Confirmados,Poblacion,Casos Activos,fallecidos,geometry
0,Antofagasta,2020-04-13,2.0,21.536336,149.0,691854.0,78.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
1,Antofagasta,2020-04-15,2.0,25.438893,176.0,691854.0,88.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Antofagasta,2020-04-17,2.0,30.497764,211.0,691854.0,103.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
3,Antofagasta,2020-04-20,2.0,38.158340,264.0,691854.0,129.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
4,Antofagasta,2020-04-24,2.0,53.190413,368.0,691854.0,172.0,2.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
...,...,...,...,...,...,...,...,...,...
787,Maule,2020-11-13,7.0,1833.225995,20751.0,1131939.0,580.0,447.0,"POLYGON ((-8018823.211 -4121471.456, -8018820...."
788,Maule,2020-11-16,7.0,1855.135303,20999.0,1131939.0,552.0,453.0,"POLYGON ((-8018823.211 -4121471.456, -8018820...."
789,Maule,2020-11-20,7.0,1880.313338,21284.0,1131939.0,600.0,461.0,"POLYGON ((-8018823.211 -4121471.456, -8018820...."
790,Maule,2020-11-23,7.0,1900.014047,21507.0,1131939.0,515.0,469.0,"POLYGON ((-8018823.211 -4121471.456, -8018820...."


Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [189]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [149]:
import ipywidgets as widgets
from ipywidgets import interactive, interact
casos_geo.rename(columns={0:'Poblacion'},inplace=True)

In [ ]:
col_widget =widgets.Dropdown(options=['Poblacion','Casos Confirmados','Tasa de incidendia','Casos Activos','fallecidos'],description='Number')

In [ ]:
fecha_widget = widgets.DatePicker(description='fecha',continuous_update=False)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** RESPONDE AQUÍ